In [1]:
import cv2 as cv
import numpy as np
import skimage.io
import matplotlib.pyplot as plt
%matplotlib inline
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [4]:
import pandas as pd

In [2]:
def plot_image(img, cmap='gray'):
    fig, ax = plt.subplots(figsize=(10, 10))
    ax.imshow(img, cmap=cmap)
    plt.show()

In [19]:
df = pd.read_pickle('data/df.pkl')
final_df = pd.read_csv('data/final_df.csv')

In [70]:
from torchvision.models import resnet18
from torchvision.datasets import EMNIST
from torchvision import transforms
from torch import nn
from torch.utils.data import DataLoader
import torch
import torch.nn.functional as F

In [93]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x)

In [94]:
model = Net()
model.load_state_dict(torch.load('data/model.pth'))
model.eval()

Net(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2_drop): Dropout2d(p=0.5, inplace=False)
  (fc1): Linear(in_features=320, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
)

In [104]:
classes = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'J', 'K', 'Q']
classes_map = {'0':0, '1':1, '2':2, '3':3, '4':4, '5':5, '6':6, '7':7, '8':8, '9':9, 'J':10, 'K':11, 'Q':12}
pred_dict = {c1:{c2:0 for c2 in classes} for c1 in classes}

In [108]:
from tqdm import tqdm

X = []
y = []

with torch.no_grad():
    for i in tqdm(range(91)):

        data_row = df.iloc[i]
        truth_row = final_df.iloc[i]
        cards = ['P1_number', 'P2_number', 'P3_number', 'P4_number']
        
        for idx in cards:
            vector = cv.resize(data_row[idx], (28, 28))
            card = torch.tensor(vector, dtype=torch.float).unsqueeze(0).unsqueeze(0)
            truth = truth_row[idx]    
            out = model(card)
            X.append(out.numpy().reshape(-1))
            y.append(classes_map[truth])
            _, predicted_tensor = torch.max(out.data, 1)
            predicted = predicted_tensor.item()
            
            pred_dict[str(predicted)][truth] += 1

100%|██████████| 91/91 [00:00<00:00, 363.03it/s]


In [97]:
df_pred = pd.DataFrame(pred_dict)

In [98]:
df_pred

,0,1,2,3,4,5,6,7,8,9,J,K,Q
0,24,2,2,0,0,0,0,0,0,0,0,0,0
1,0,24,0,0,0,0,0,3,1,0,0,0,0
2,2,0,25,0,0,0,0,1,0,0,0,0,0
3,0,0,0,27,0,0,0,1,1,0,0,0,0
4,0,0,0,0,26,0,1,1,0,0,0,0,0
5,0,0,1,2,0,25,0,0,0,0,0,0,0
6,1,3,0,0,0,0,27,0,1,0,0,0,0
7,0,1,3,0,0,0,0,22,0,0,0,0,0
8,0,0,0,1,0,0,0,1,26,0,0,0,0
9,0,0,1,1,0,0,0,5,6,11,0,0,0


In [102]:
precision_vector = df_pred.apply(collapse_axis, axis=0)
recall_vector = df_pred.apply(collapse_axis, axis=1)

/home/nabegh/Anaconda3/envs/kaggle/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in long_scalars
  


In [103]:
print("Precision: ", precision_vector)
print("Recall: ", recall_vector)

Precision:  0    0.888889
1    0.571429
2    0.714286
3    0.870968
4    0.742857
5    1.000000
6    0.375000
7    0.611111
8    0.520000
9    1.000000
J         NaN
K         NaN
Q         NaN
dtype: float64
Recall:  0    0.857143
1    0.857143
2    0.892857
3    0.931034
4    0.928571
5    0.892857
6    0.843750
7    0.846154
8    0.928571
9    0.458333
J    0.000000
K    0.000000
Q    0.000000
dtype: float64


In [101]:
def collapse_axis(row):
    return row[row.name] / row.sum()
    

In [109]:
X = np.array(X)
y = np.array(y)

In [134]:
pred_dict = {i:{j:0 for j in range(13)} for i in range(13)}

In [126]:
import xgboost
from sklearn.model_selection import KFold, cross_validate
from sklearn.model_selection import cross_val_score

In [123]:
model = xgboost.XGBClassifier()
kfold = KFold(n_splits=5, shuffle=True, random_state=107)
results = cross_val_score(model, X, y, cv=kfold)
print("Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

/home/nabegh/Anaconda3/envs/kaggle/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[21:11:48] WARNING: /tmp/build/80754af9/xgboost-split_1619724447847/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[21:11:49] WARNING: /tmp/build/80754af9/xgboost-split_1619724447847/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/home/nabegh/Anaconda3/envs/kaggle/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/nabegh/Anaconda3/envs/kaggle/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[21:11:49] WARNING: /tmp/build/80754af9/xgboost-split_1619724447847/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[21:11:49] WARNING: /tmp/build/80754af9/xgboost-split_1619724447847/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/home/nabegh/Anaconda3/envs/kaggle/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/nabegh/Anaconda3/envs/kaggle/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[21:11:49] WARNING: /tmp/build/80754af9/xgboost-split_1619724447847/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Accuracy: 84.63% (4.06%)


In [136]:
kfold = KFold(n_splits=5, shuffle=True, random_state=7)
for train_index, test_index in kfold.split(range(X.shape[0])):
    model = xgboost.XGBClassifier()
    X_train = X[train_index, :]
    y_train = y[train_index]
    X_test = X[test_index, :]
    y_test = y[test_index]
    
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    for i in range(len(y_pred)):
         pred_dict[y_pred[i]][y_test[i]] += 1

[21:20:38] WARNING: /tmp/build/80754af9/xgboost-split_1619724447847/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[21:20:38] WARNING: /tmp/build/80754af9/xgboost-split_1619724447847/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[21:20:38] WARNING: /tmp/build/80754af9/xgboost-split_1619724447847/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[21:20:38] WARNING: /tmp/build/80754af9/xgboost-split_1619724447847/work/src/learner.cc:106

In [137]:
df_pred = pd.DataFrame(pred_dict)

In [140]:
precision_vector = df_pred.apply(collapse_axis, axis=0).values
recall_vector = df_pred.apply(collapse_axis, axis=1).values

In [141]:
print("Precision: ", precision_vector)
print("Recall: ", recall_vector)

Precision:  [0.92307692 0.75862069 0.82142857 0.93103448 0.92592593 0.92857143
 0.96428571 0.8        0.83333333 0.61538462 0.8125     0.9
 0.92307692]
Recall:  [0.85714286 0.78571429 0.82142857 0.93103448 0.89285714 0.92857143
 0.84375    0.76923077 0.89285714 0.66666667 0.92857143 0.93103448
 0.85714286]


In [142]:
2 * (precision_vector * recall_vector) / (precision_vector + recall_vector)

array([0.88888889, 0.77192982, 0.82142857, 0.93103448, 0.90909091,
       0.92857143, 0.9       , 0.78431373, 0.86206897, 0.64      ,
       0.86666667, 0.91525424, 0.88888889])

In [143]:
df_pred

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,24,2,2,0,0,0,0,0,0,0,0,0,0
1,0,22,0,0,0,0,0,0,1,4,1,0,0
2,1,0,23,0,0,1,0,2,0,0,1,0,0
3,0,0,1,27,0,0,0,0,0,1,0,0,0
4,0,1,0,0,25,0,1,0,0,1,0,0,0
5,0,0,0,2,0,26,0,0,0,0,0,0,0
6,1,2,0,0,0,0,27,0,1,0,0,0,1
7,0,1,2,0,0,0,0,20,0,3,0,0,0
8,0,1,0,0,0,0,0,0,25,0,2,0,0
9,0,0,0,0,1,1,0,3,2,16,1,0,0
